In [19]:
from pathlib import Path
import numpy as np
from numpy import array
import h5py
import pandas as pd
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [20]:
#read data from csv
data = pd.read_csv("tessdata.csv", names=["image", "text","confidence"])

#images i.e X
X = data['image']
#target variable
y = data['confidence']

data.head(5)

,image,text,confidence
0,broodiest_183.jpg,proodiest\n,33
1,tendential gangliglia oppositeness_7617.jpg,tendential gangliglia oppositeness\n,92
2,limbate myotic authoritarians_5861.jpg,limbate myotic authoritarians\n,90
3,tiptops Hardenberg skin-deep_3153.jpg,tiptops Hardenberg skin-deep\n,91
4,brachypodine Goulder belemnitic_498.jpg,brachypodine Geulder belernnitic\n,62


In [21]:
hdf5_dir = Path("hdf5/")
# hdf5_dir.mkdir(parents=True, exist_ok=True)

In [22]:
def create_data():
    X_data, Y_data =  list(), list()  
    for i, j in data.iterrows():
        image_name, confidence = X[i], y[i]
        image_id = image_name.split('.')[0]
        filename = 'out' + '/' + image_name
        image = load_img(filename, target_size=(50,50))
        image = img_to_array(image)
        X_data.append(image)
        Y_data.append(confidence)
    X_data, Y_data = array(X_data), array(Y_data)
    return X_data, Y_data

In [23]:
X, Y = create_data()
print(X.shape, Y.shape)

(15000, 50, 50, 3) (15000,)


In [24]:
def save_images_h5():
    num_images = len(X)
    file = h5py.File(hdf5_dir / f"{num_images}_.h5", "w")
    # Create a dataset in the file
    dataset = file.create_dataset(
        "images", np.shape(X), h5py.h5t.STD_U8BE, data=X)
    
    meta_set = file.create_dataset(
        "confidence", np.shape(Y), h5py.h5t.STD_U8BE, data=Y
    )
    file.close()


In [25]:
save_images_h5()



In [26]:
def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/confidence"]).astype("uint8")

    return images, labels

X_data, Y_data = read_many_hdf5(15000)
print(X_data.shape, Y_data.shape)

(15000, 50, 50, 3) (15000,)
